In [0]:
import pyspark.sql.functions as F
from pyspark.sql.functions import *

airbnb_df = spark.table("az_adb_simbus_training.inside_air_bnb_project_schema.bronze_listings")
display(airbnb_df.limit(10))


# Basic Preprocessing

Removing Null Coulmns

In [0]:
airbnb_df = airbnb_df.drop('neighbourhood_group', 'license')

Checking remaining null count

In [0]:
from pyspark.sql.functions import col, sum as _sum, when
from pyspark.sql import Row

# Calculate nulls for each column
null_counts = airbnb_df.select([
    _sum(when(col(c).isNull(), 1).otherwise(0)).alias(c)
    for c in airbnb_df.columns
]).collect()[0].asDict()

# Convert to a DataFrame like pandas-style output
null_df = spark.createDataFrame([Row(column=k, null_count=v) for k, v in null_counts.items()])
null_df.show()


In [0]:
pandas_df = airbnb_df.toPandas()

In [0]:
import missingno as msno
import matplotlib.pyplot as plt

# Basic nullity matrix
msno.matrix(pandas_df)
plt.show()

In [0]:
airbnb_df = airbnb_df.na.drop(how='any')

In [0]:
from pyspark.sql.functions import col, sum as _sum, when
from pyspark.sql import Row

null_counts = airbnb_df.select([
    _sum(when(col(c).isNull(), 1).otherwise(0)).alias(c)
    for c in airbnb_df.columns
]).collect()[0].asDict()

null_df = spark.createDataFrame([Row(column=k, null_count=v) for k, v in null_counts.items()])
null_df.show()


In [0]:
display(airbnb_df.limit(10))

In [0]:
desc_pdf = airbnb_df.describe().toPandas()

# Transpose and set proper headers
desc_pdf = desc_pdf.set_index('summary').T.reset_index().rename(columns={'index': 'column'})

display(desc_pdf)

In [0]:
import re
from pyspark.sql.functions import udf, col
from pyspark.sql.types import StringType

whitelist_tokens = {
    "p/", "p/mo", "p/month", "p/wk", "p/week", "p/day", "p/sq ft",
    "sq ft", "sq. ft.", "sq m", "br", "ba", "w/d", "a/c", "hwd", "apt",
    "unit", "ft", "hr", "mo", "rm", "dep", "incl", "util", "pkg", "ref",
    "elec", "gas", "hwh"
}

non_alphanum_pattern = re.compile(r"[^a-zA-Z0-9/]")

def strict_clean_text(text):
    if text is None:
        return None

    text = re.sub(r"<.*?>", " ", text) #html tags

    tokens = text.split()
    cleaned_tokens = []

    for token in tokens:
        token_lower = token.lower()
        if token_lower in whitelist_tokens:
            cleaned_tokens.append(token)
        else:
            cleaned = re.sub(non_alphanum_pattern, " ", token)
            if cleaned:
                cleaned_tokens.append(cleaned)

    return " ".join(cleaned_tokens)

strict_clean_text_udf = udf(strict_clean_text, StringType())
airbnb_df = airbnb_df.withColumn("name_cleaned", strict_clean_text_udf(col("name")))


In [0]:
display(airbnb_df.limit(10))

In [0]:
from pyspark.sql.functions import countDistinct
from pyspark.sql import Row

unique_counts_row = airbnb_df.agg(*[
    countDistinct(col_name).alias(col_name) for col_name in airbnb_df.columns
]).collect()[0]

unique_counts_list = [
    Row(column=col_name, unique_count=unique_counts_row[col_name])
    for col_name in airbnb_df.columns
]

unique_counts_df = spark.createDataFrame(unique_counts_list)
unique_counts_df.orderBy("unique_count", ascending=False).show(truncate=False)


In [0]:
from pyspark.sql.functions import to_date

airbnb_df = airbnb_df.withColumn("last_review", to_date("last_review", "yyyy-MM-dd"))


In [0]:
final_df_with_names.write.format("delta").mode("overwrite").saveAsTable("az_adb_simbus_training.inside_air_bnb_project_schema.silver_listing")


In [0]:
. Monthly Price Aggregates
Columns:
- listing_id
- neighborhood
- year
- month
- avg_price

In [0]:
Monthly Occupancy Rates
Columns:
- listing_id
- neighborhood
- year
- month
- occupancy_rate

In [0]:
Average Rating per Listing
Columns:
- listing_id
- avg_rating

In [0]:
4. Host Features
Columns:
- host_id
- is_superhost
- response_time
- total_listings
- avg_rating (from reviews)
- avg_price (from listings)
- commidites offer(hard to do)/text extraction